## Setting up and running optimization in Jupyter


In [1]:
from geecs_scanner.data_acquisition.types import ScanConfig, ScanMode
from geecs_scanner.data_acquisition.default_scan_manager import get_default_scan_manager
from pathlib import Path

# load the default experiment configuration for Undulator (shot control, etc.)
manager = get_default_scan_manager("Undulator")

# Define the root path to the GEECS-Scanner-GUI repo
base_path = Path(
    "C:/Users/loasis.loasis/Documents/Github/GEECS-Plugins/GEECS-Scanner-GUI"
)

# Define path to the optimizer config. See documentation for optimizer config structure
optimizer_config_path = (
    base_path
    / "geecs_scanner"
    / "optimization"
    / "optimizer_configs"
    / "Undulator"
    / "ALine3_FWHM.yaml"
)

path config ../../user data/Configurations.INI


2025-05-31 19:43:30,926 - root - WARNING - wavekit not installed properly


Currently, this is the general format for an optimizer config. 

**VOCS:** define variables to be controlled 

**evaluator:** points the the function to be used to determine the value of the objective function

**generator:** name of the generator to use, which must be defined in the generator_factory

**device_requirements:** List of devices to save and use during optimzation

```python
vocs:
  variables:
    U_ESP_JetXYZ:Position.Axis 2: [ -8, -7 ]
    U_ESP_JetXYZ:Position.Axis 3: [ 2, 8]
    U_HP_Daq:AnalogOutput.Channel 1: [ 1.5, 2.5]
    U_ModeImagerESP:Position.Axis 2: [-17.2,-17.0]
  objectives:
    f: MINIMIZE
  constraints: {}

evaluator:
  module: geecs_scanner.optimization.evaluators.HiResMagCam
  class: HiResMagCam

generator:
  name: bayes_default

device_requirements:
  Devices:
    UC_HiResMagCam:
      add_all_variables: false
      save_nonscalar_data: true
      synchronous: true
      variable_list:
        - acq_timestamp
    U_BCaveICT:
      add_all_variables: false
      save_nonscalar_data: true
      synchronous: true
      variable_list:
        - acq_timestamp
        - Python Results.ChA
    U_BCaveMagSpec:
      add_all_variables: false
      save_nonscalar_data: true
      synchronous: true
      variable_list:
        - acq_timestamp
```

In [ ]:
# success = manager.reinitialize(config_path=config_filename)
success = manager.reinitialize()

# manager.shot_control_variables['Trigger.Source']['OFF']= 'Single shot'
# manager.shot_control_variables['Trigger.Source']['SCAN']= 'Internal'
# manager.save_local = False  # If true, will save locally on device PC before being queued to transfer to network


scan_config = ScanConfig
scan_config.start = 0
scan_config.end = 5
scan_config.step = 1
scan_config.wait_time = 3
scan_config.scan_mode = ScanMode.OPTIMIZATION
scan_config.optimizer_config_path = optimizer_config_path

# scan_config_dict = {
#     'start': 0,
#     'end': 0,
#     'step': 1,
#     'wait_time': 5.5,
#     'additional_description': 'Testing out new python data acquisition module',
#     'scan_mode': ScanMode.NOSCAN}
# scan_config=ScanConfig(**scan_config_dict)

In [1]:
from geecs_scanner.data_acquisition.types import ScanConfig, ScanMode
from geecs_scanner.data_acquisition.default_scan_manager import get_default_scan_manager
from pathlib import Path


manager = get_default_scan_manager("Undulator")
base_path = Path(
    "/Users/samuelbarber/Desktop/Github_repos/GEECS-Plugins/GEECS-Scanner-GUI"
)
base_path = Path(
    "C:/Users/loasis.loasis/Documents/Github/GEECS-Plugins/GEECS-Scanner-GUI"
)

config_filename = (
    base_path
    / "scanner_configs"
    / "experiments"
    / "Undulator"
    / "save_devices"
    / "UC_TC_Phosphor.yaml"
)

optimizer_config_path = (
    base_path
    / "geecs_scanner"
    / "optimization"
    / "optimizer_configs"
    / "Undulator"
    / "ALine3_FWHM.yaml"
)


# success = manager.reinitialize(config_path=config_filename)
success = manager.reinitialize()

# manager.shot_control_variables['Trigger.Source']['OFF']= 'Single shot'
# manager.shot_control_variables['Trigger.Source']['SCAN']= 'Internal'
# manager.save_local = False  # If true, will save locally on device PC before being queued to transfer to network


scan_config = ScanConfig
scan_config.start = 0
scan_config.end = 5
scan_config.step = 1
scan_config.wait_time = 3
scan_config.scan_mode = ScanMode.OPTIMIZATION
scan_config.optimizer_config_path = optimizer_config_path

# scan_config_dict = {
#     'start': 0,
#     'end': 0,
#     'step': 1,
#     'wait_time': 5.5,
#     'additional_description': 'Testing out new python data acquisition module',
#     'scan_mode': ScanMode.NOSCAN}
# scan_config=ScanConfig(**scan_config_dict)

path config ..\..\user data\Configurations.INI


C:\Users\loasis.LOASIS\AppData\Local\Temp\ipykernel_9028\1097318967.py:2: DeprecationWarning: geecs_scanner.data_acquisition.types.ScanConfg/ScanMode have moved to geecs_data_utils.types; please update your imports.
  from geecs_scanner.data_acquisition.types import ScanConfig, ScanMode
2025-06-11 22:16:44,925 - root - INFO - Loaded composite variables from C:\Users\loasis.LOASIS\Documents\GitHub\GEECS-Plugins\GEECS-Scanner-GUI\scanner_configs\experiments\Undulator\scan_devices\composite_variables.yaml
2025-06-11 22:16:44,952 - root - INFO - Loaded master actions from C:\Users\loasis.LOASIS\Documents\GitHub\GEECS-Plugins\GEECS-Scanner-GUI\scanner_configs\experiments\Undulator\action_library\actions.yaml
2025-06-11 22:16:44,976 [INFO] Logging session started.
2025-06-11 22:16:44,977 [INFO] constructing the scan step executor
2025-06-11 22:16:44,978 [INFO] synchronous variables after reset: []
2025-06-11 22:16:44,979 [INFO] asynchronous variables after reset: []
2025-06-11 22:16:44,979 [

Logging has been stopped and handlers have been removed.


In [2]:
manager.scan_config = scan_config
manager.start_scan_thread(scan_config)

2025-06-11 22:16:51,999 [INFO] scan config is this: <class 'geecs_data_utils.type_defs.ScanConfig'>
2025-06-11 22:16:52,002 [INFO] scan config getting sent to pre logging is this: <class 'geecs_data_utils.type_defs.ScanConfig'>
2025-06-11 22:16:52,004 [INFO] Scan thread started.
2025-06-11 22:16:52,004 [INFO] Turning off the trigger.
2025-06-11 22:16:52,183 [INFO] Setting Amplitude.Ch AB to 0.5
2025-06-11 22:16:52,436 [INFO] Setting Trigger.Source to Single shot
2025-06-11 22:16:52,437 [INFO] Trigger turned to state OFF.
2025-06-11 22:16:52,452 [INFO] Created folder: Z:\data\Undulator\Y2025\06-Jun\25_0611\scans\Scan004
2025-06-11 22:16:52,456 [INFO] TDMS writer initialized with path: Z:\data\Undulator\Y2025\06-Jun\25_0611\scans\Scan004\Scan004.tdms


Successfully loaded config file: C:\Users\loasis.LOASIS\Documents\GitHub\GEECS-Plugins\LogMaker4GoogleDocs\logmaker_4_googledocs\config.ini


2025-06-11 22:16:55,017 [INFO] Initialized generator expected_improvement
2025-06-11 22:16:55,022 [INFO] config dict is {'Devices': {'UC_ALineEBeam3': {'add_all_variables': False, 'save_nonscalar_data': True, 'synchronous': True, 'variable_list': ['acq_timestamp']}}}
2025-06-11 22:16:55,023 [INFO] validated SaveDeviceConfig is Devices={'UC_ALineEBeam3': DeviceConfig(synchronous=True, save_nonscalar_data=True, variable_list=['acq_timestamp'], post_analysis_class=None, scan_setup=None)} scan_info='additional description' setup_action=None closeout_action=None
2025-06-11 22:16:55,024 [INFO] UC_ALineEBeam3: Synchronous = True, Save_Non_Scalar = True
2025-06-11 22:16:55,045 [INFO] Subscribing UC_ALineEBeam3 to variables: ['acq_timestamp']
2025-06-11 22:16:55,047 [INFO] Devices loaded: dict_keys(['UC_ALineEBeam3'])
2025-06-11 22:16:55,048 [INFO] Loaded scan info: 
2025-06-11 22:16:55,048 [INFO] Adding new scan device: U_EMQTripletBipolar with default settings.
2025-06-11 22:16:55,069 [INFO] 

Logging has been stopped and handlers have been removed.
Attempting to move scan_execution.log to Z:\data\Undulator\Y2025\06-Jun\25_0611\scans\Scan004\scan_execution.log
Moved log file to Z:\data\Undulator\Y2025\06-Jun\25_0611\scans\Scan004\scan_execution.log


INFO:root:Successfully unsubscribed from UC_ALineEBeam3.
INFO:root:Attempting to unsubscribe from U_EMQTripletBipolar...
INFO:root:Successfully unsubscribed from U_EMQTripletBipolar.
INFO:root:synchronous variables after reset: []
INFO:root:asynchronous variables after reset: []
INFO:root:non_scalar_saving_devices devices after reset: []
INFO:root:devices devices after reset: {}
INFO:root:DeviceManager instance has been reset and is ready for reinitialization.
